In [3]:
# Import panadas as pd
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [4]:
# Download NLTK resources
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
# 2. Load dataset
df = pd.read_csv("/content/spam.csv", encoding='latin-1')

In [10]:
print(df.columns)

Index(['Category', 'Message'], dtype='object')


In [11]:
# Clean column names
df = df[['Category', 'Message']]
df.columns = ['label', 'message']

In [13]:
# Preprocess text
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
  text = text.lower()
  text = ''.join([c for c in text if c not in string.punctuation])
  words = text.split()
  words = [stemmer.stem(word) for word in words if word not in stop_words]
  return ' '.join(words)

df['clean_message'] = df['message'].apply(preprocess)

In [14]:
# Convert labels to binary
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})

In [15]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(df['clean_message'],df['label_num'], test_size=0.2, random_state=42)

In [21]:
# Fit on training data only
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)

In [22]:
# Use the same vectorizer to transform test data
X_test_tfidf = vectorizer.transform(X_test)

In [23]:
# Train the model
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

MultinomialNB()

In [24]:
# Predict
y_pred = model.predict(X_test_tfidf)

In [25]:
# 9. Evaluation
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Confusion Matrix:
 [[966   0]
 [ 37 112]]

Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       966
           1       1.00      0.75      0.86       149

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115

Accuracy: 0.9668161434977578


In [28]:
# Test your own email
test_email = "Hello i call you later"
test_email_clean = preprocess(test_email)
test_email_vector = vectorizer.transform([test_email_clean])
prediction = model.predict(test_email_vector)

# Output
if prediction[0] == 1:
    print("This email is SPAM.")
else:
    print("This email is NOT SPAM.")

🟢 This email is NOT SPAM.
